# Initializing the SysML v2 API

In [ ]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json

#host = "<SysML2-API-Repo-Host:Port>"
host = "Specify the URL of the SysML v2 API and Service Repository"

# Get projects

In [ ]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    
    df = pd.DataFrame({'Project Name':[], 'Project ID':[]})
    for p in projects:
        df = pd.concat([df, pd.DataFrame({'Project Name':[p['name']], 'Project ID':[p['@id']]})], ignore_index=True)
    display(df)

# Define a function to recursively iterate over feature members 
## Recursive graph traversal using FeatureMembership relations

In [ ]:
def get_member_features(project_id, commit_id, element_id, member_type, indent):
    
    # Fetch the element
    element_url = f"{host}/projects/{project_id}/commits/{commit_id}/elements/{element_id}" 
    response = requests.get(element_url)
    
    element_data = response.json()
    element_name = element_data['name']
    element_id = element_data['@id']
    element_type = element_data ['@type']
    
    if element_type == member_type:
        print(f"{indent} - {element_name} (id = {element_id}, type = {element_type})")
        # Feature memberships
        element_features = element_data['feature']
        if len(element_features) > 0:
            for feature in element_features:
                get_member_features(project_id, commit_id, feature['@id'], member_type, indent + "  ")


# Parts tree navigation

In [ ]:
parts_tree_project_id = '85d874ae-1f68-4949-9b16-378f1cbc477e'
parts_tree_commit_id = ''

commits_url = f"{host}/projects/{parts_tree_project_id}/commits" 
commits_response = requests.get(commits_url)
if commits_response.status_code == 200:
    commits = commits_response.json()
    pprint(commits)
    parts_tree_commit_id = commits[0]['@id']


## Query to find vehicle1

In [ ]:
# Query vehicle1 (PartUsage) in the 
data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'vehicle1'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'PartUsage'
        }
    ]
  }
}

payload = json.dumps(data)

vehicle1_id = ''
query_url = f"{host}/projects/{parts_tree_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    vehicle1_id = query_response_json[0]['@id']

## Parts tree for vehicle1 (recursive FeatureMembership relation navigation)

In [ ]:
get_member_features(parts_tree_project_id,parts_tree_commit_id,vehicle1_id,'PartUsage'," ")

## Query to find vehicle1_c1

In [ ]:

data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'vehicle1_c1'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'PartUsage'
        }
    ]
  }
}
payload = json.dumps(data)

vehicle1_c1_id = ''
query_url = f"{host}/projects/{parts_tree_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    vehicle1_c1_id = query_response_json[0]['@id']

## Parts tree for vehicle1_c1 (recursive FeatureMembership relation navigation)

In [ ]:
get_member_features(parts_tree_project_id,parts_tree_commit_id,vehicle1_c1_id,'PartUsage'," ")

# Behavior Tree Navigation

In [ ]:
behavior_project_id = '128710cd-978f-4c6f-93ac-8afbbd194185'
behavior_project_commit_id = ''

commits_url = f"{host}/projects/{behavior_project_id}/commits" 
commits_response = requests.get(commits_url)
if commits_response.status_code == 200:
    commits = commits_response.json()
    pprint(commits)
    behavior_project_commit_id = commits[0]['@id']

## Query to find Provide Power behavior (ActionUsage)

In [ ]:
data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'provide power'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'ActionUsage'
        }
    ]
  }
}

payload = json.dumps(data)

provide_power_id = ''
query_url = f"{host}/projects/{behavior_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    provide_power_id = query_response_json[0]['@id']

## Behavior tree for vehicle1 (recursive FeatureMembership relation navigation)

In [ ]:
get_member_features(behavior_project_id, behavior_project_commit_id, provide_power_id,"ActionUsage"," ")

# Requirement tree for vehicle

In [ ]:
simple_vehicle_model_project = '8b377f03-9f29-4ae9-b60b-d266dc4a8180'
vehicle_commit = '10f84681-3d17-43be-b59a-89c480d1abeb'

In [ ]:
data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'vehicleSpecification'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'RequirementUsage'
        }
    ]
  }
}

payload = json.dumps(data)

vehicle_specification_id = ''
query_url = f"{host}/projects/{simple_vehicle_model_project}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    vehicle_specification_id = query_response_json[0]['@id']

In [ ]:
get_member_features(simple_vehicle_model_project, vehicle_commit, vehicle_specification_id,"RequirementUsage"," ")